# The McCall Search Model

In [12]:
using LinearAlgebra

We must first create a function for the Bellman mapping.

In [30]:
"""
    mccallbellmanmap(v,w,π,c,β)

Iterates the McCall search model bellman equation for with value function v.
Returns the new value function.

# Arguments
* `v` vector of values for each wage
* `w` vector of wages
* `p` vector of probabilities for each wage
* `c` unemployment benefits
* `β` time discount factor

"""

function mccallbellmanmap(v, w, p, c, β)
    
    # create empty vector for value storage
    v_out = zeros(length(w))
    
    # compute value given each wage in vector `w`
    for i in 1:length(w)
        
        # Bellman map: v = max{c + β E(v'), w/(1-β)}
        v_out[i] = max(c + β * dot(p,v), w[i]/(1-β)) 
        
    end
    
    return v_out
end

mccallbellmanmap (generic function with 1 method)

Alternatively:

In [31]:
"""
    mccallbellmanmap_v2(v,w,π,c,β)

Iterates the McCall search model bellman equation for with value function v.
Returns the new value function.

# Arguments
* `v` vector of values for each wage
* `w` vector of wages
* `p` vector of probabilities for each wage
* `c` unemployment benefits
* `β` time discount factor

"""
function mccallbellmanmap_v2(v, w, p, c, β)
    
    # compute value of rejecting the offer
    v_reject = c + β * dot(p,v) # this is a Float
    
    # compute value of accepting the wage offer
    v_accept = w / (1-β)
    
    # compute v_reject to v_accept
    v_out = max.(v_reject, v_accept)
    
    return v_out
end

mccallbellmanmap_v2

We use the Bellman map to generate a sequence of vectors of $v(w_t)$ for "all" $t$ and all possible realizations of $w$..

In [33]:
# Create discrete uniform wage distribution between 1 and 10
# where S = # of grid points (evenly-spaced points between 1 and 10)
S = 5
w = LinRange(1, 10, S) 
p = ones(S)/S # each p[i] = 1/S ∀i, so that p(w[i]) = p(w[j]) ∀i,j ∈ {1,...,S}

# Set model parameters:
β = 0.95 # note that lower β causes code to converge faster
c = 3 # unemployment wage/benefits

# Initiate the value function
v0 = zeros(S)

# Iterate code J times
J = 50

# Create and initiate array containing 
# iterated value function values:
V = zeros(J, S)
V[1,:] = mccallbellmanmap(v0, w, p, c, β)

# Keep applying Bellman map recursively 
for j in 2:J
    V[j,:] = mccallbellmanmap_v2(V[j-1,:], w, p, c, β)
end

V

50×5 Array{Float64,2}:
  20.0     65.0    110.0    155.0    200.0
 107.5    107.5    110.0    155.0    200.0
 132.2    132.2    132.2    155.0    200.0
 145.804  145.804  145.804  155.0    200.0
 153.558  153.558  153.558  155.0    200.0
 157.978  157.978  157.978  157.978  200.0
 161.063  161.063  161.063  161.063  200.0
 163.408  163.408  163.408  163.408  200.0
 165.19   165.19   165.19   165.19   200.0
 166.545  166.545  166.545  166.545  200.0
 167.574  167.574  167.574  167.574  200.0
 168.356  168.356  168.356  168.356  200.0
 168.951  168.951  168.951  168.951  200.0
   ⋮                                 
 170.832  170.832  170.832  170.832  200.0
 170.832  170.832  170.832  170.832  200.0
 170.832  170.832  170.832  170.832  200.0
 170.833  170.833  170.833  170.833  200.0
 170.833  170.833  170.833  170.833  200.0
 170.833  170.833  170.833  170.833  200.0
 170.833  170.833  170.833  170.833  200.0
 170.833  170.833  170.833  170.833  200.0
 170.833  170.833  170.833  170.833 

In [34]:
"""
    solvemccall(w,π,c,β[,ϵ])

Iterates the McCall search model bellman equation until convergence criterion 
ϵ is reached

# Arguments
* `w` vector of wages
* `p` vector of probabilities for each wage
* `c` unemployment benefits
* `β` time discount factor
* `ϵ' Stopping criteria (default 1e-6)
"""

function solvemccall(w,p,c,β,ϵ=1e-6)
    #initialize
    v = w/(1-β)
    diff = 1.
    #check if stoping criteria is reached
    while diff > ϵ
        v_new = mccallbellmanmap(v,w,p,c,β)
        #use supremum norm
        diff = norm(v-v_new,Inf)
        v = v_new #reset v
    end
    return v
end

solvemccall (generic function with 2 methods)

In [10]:
v = solvemccall(w,p,c,β)
println(v - mccallbellmanmap(v,w,p,c,β))

[-7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, -7.001465860412281e-7, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
